In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from tqdm import tqdm_notebook
from PIL import Image

In [11]:
%run config.ipynb
#from config import *
%run data.ipynb
#from data import *
%run generate_image_model.ipynb
#from generate_image_model import *
%run sequence_model.ipynb
#from sequence_model import *
%run joint_model.ipynb
#from joint_model import *
%run utils.ipynb
#from utils import *

In [4]:
# generate data loader
train_loader = get_loader(train=True, image=IMAGE_MODE, sequence=SEQUENCE_MODE)
val_loader = get_loader(val=True, image=IMAGE_MODE, sequence=SEQUENCE_MODE)
test_loader = get_loader(test=True, image=IMAGE_MODE, sequence=SEQUENCE_MODE)

In [12]:
# Tensorboard
from tensorboardX import SummaryWriter
print('Experiment Setting: ', model_name)
writer = SummaryWriter('runs/'+model_name+('_test' if Test_Mode else '_train'))

('Experiment Setting: ', 'models/generation_baseline')


In [13]:
def main():
    # model
    Net = GenerateNet
    model = Net().cuda()
    logs = torch.load(Test_Model)
    model.classification.load_state_dict(logs['weights'])
    
    # optimizer and scheduler
    optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad], lr=initial_lr)
    scheduler = lr_scheduler.ExponentialLR(optimizer, 0.5**(1 / lr_halflife))
    start_epoch = 0
    
    if Test_Mode:
        resume = torch.load(Test_Model)
        model.load_state_dict(resume['weights'])
        test_ac = train(model, test_loader, optimizer, scheduler, resume['epoch'], train=False)
        return
    
    for i in range(num_epoch):
        #'Training epoch: '
        train_ac = train(model, train_loader, optimizer, scheduler, i, train=True)
        #'Evaluation epoch: '
        val_ac = train(model, val_loader, optimizer, scheduler, i, train=False)
        
        #save model
        results = {
            'epoch':i,
            'weights':model.state_dict(),
            'training accuracy': train_ac,
            'eval accuracy':val_ac,
        }
        torch.save(results, model_name + '_' + str(i) + '.pth')
        

In [18]:
def train(model, loader, optimizer, scheduler, epoch, train=False):
    if train:
        model.train()
        model.classification.eval()
    else:
        model.eval()
        model.classification.eval()
    
     # container
    output_container = []
    loss_container = []
    
    loader = tqdm_notebook(loader, desc='Training epoch '+str(epoch) if train else 'Eval epoch '+str(epoch))
    batch_max = len(loader)
    batch_count = 0
    for v, s, l in loader:
        var_params = {
            'requires_grad': False,
        }
        v = Variable(v.cuda(async=True), **var_params) # image
        s = Variable(s.cuda(async=True), **var_params) # sequence
        l = Variable(l.cuda(async=True), **var_params) # label
        
        draws, out, l2_loss = model(l)
        if batch_count%200 == 0:
            label = int(l[0])
            name = 'images/' + str(batch_count) + '_' + str(label) + '.png'
            name_org = 'images/' + str(batch_count) + '_' + str(label) + '_org' + '.png'
            single_draw = draws[0].view(28,28).data.cpu().numpy().astype(dtype = np.uint8)
            image_draw = Image.fromarray(single_draw, 'P')
            image_draw.save(name)
            
            single_org = v[0].view(28,28).data.cpu().numpy().astype(dtype = np.uint8)
            image_org = Image.fromarray(single_org, 'P')
            image_org.save(name_org)
        if train:
            loss = F.cross_entropy(out, l) + l2_loss * 0.0001
            acc = (l == out.max(1)[1]).float().mean()
            output_container.append(float(acc.detach().data))
            loss_container.append(float(loss.detach().data))
            scheduler.step()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # Add Tensorboard log
            add_log(writer, epoch*batch_max+batch_count, loss_container, output_container, train=True)
        else:
            acc = (l == out.max(1)[1]).float().mean()
            output_container.append(float(acc))
            # Add Tensorboard log
            add_log(writer, epoch*batch_max+batch_count, loss_container, output_container, test=True)
            
        # visualization
        loader.set_postfix(loss=list_mean(loss_container), acc=list_mean(output_container))
        
        batch_count+=1
    
    return list_mean(output_container)

In [19]:
main()

SEJveChjaGlsZHJlbj0oSW50UHJvZ3Jlc3ModmFsdWU9MCwgZGVzY3JpcHRpb249dSdUcmFpbmluZyBlcG9jaCAwJywgbWF4PTI1MTQxLCBzdHlsZT1Qcm9ncmVzc1N0eWxlKGRlc2NyaXB0aW/igKY=


Process Process-19:
Process Process-17:
Process Process-20:
Process Process-18:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/kaihua/anaconda3/envs/course/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
  File "/home/kaihua/anaconda3/envs/course/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
  File "/home/kaihua/anaconda3/envs/course/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
  File "/home/kaihua/anaconda3/envs/course/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
    self.run()
  File "/home/kaihua/anaconda3/envs/course/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/kaihua/anaconda3/envs/course/lib/python2.7/multiprocessing/process.py", line 114, in run
    self.run()
    self.run()
    self._target(*self._args, **self._kwargs)
    self._target(*self._args

KeyboardInterrupt: 